In [1]:
from diskcache_class import db
from lda_mallet import *
from pre_processing import *
from os import walk
from tqdm import tqdm

[nltk_data] Downloading package stopwords to /Users/das-
[nltk_data]     lab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/das-lab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/das-lab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/das-lab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [53]:
#we are going to run this on 4 different dataset 1. EDML 2. AP 3. Newsgroup 4.Covid-Tweet


############################## reading data for ED corpus
# datafolder = './data/ed_recovery_formatted/Excel'
# #datafolder = 'ed_recovery_topicmodel'
# df = pd.DataFrame(columns=['url','type','photo','date','tags','notes','text','photo_url','reblogged','blog_name'])

# for dirpath,dirnames,filenames in walk(datafolder):
#   for filename in filenames:
#     if filename.endswith('.xlsx'):
#       t = pd.read_excel(datafolder+'/'+filename,names=['url','type','photo','date','tags','notes','text','photo_url','reblogged'])
#       blog_name = t.iloc[0,0].split(':')[1]
#       t['blog_name'] = blog_name
#       df = df.append(t.iloc[3:,:],ignore_index=True)
#       print('blog:{0}   with posts:{1}    and reblogs:{2}  '.format(filename,len(t),len(t[t.reblogged=='yes'])))


# print('number of blogs: {0} - number of posts: {1}'.format(len(set(df.blog_name)),len(df)))
# print('out of {0} documents, {1} are reblogged.'.format(len(df),len(df[df.reblogged == 'yes'])))

# #finding reblogged texts
# texts = sorted(df.text) #sort them to keep smallest post (perhaps original one) at first
# re_texts = []

# while len(texts):
#   t = [texts.pop(0)]#pop first text and find it!
#   if t[0] == ' ' or len(t[0].split())<3: 
#     continue #almost nothing to look
#   i = 0
#   while i<len(texts):
#     if t[0] in texts[i]:
#       t.append(texts.pop(i))
#     else:
#       i += 1
#   if len(t) > 1:
#     re_texts.append(t)


# print('number of unique reblogged texts: {0}'.format(len(re_texts)))
# print('number of unique string in all texts: {0}'.format(len(set(df.text))))

# extra_stopwords = ['isnt','want','cant','wanna','im','could','ive','would','dont','get','also','us','thats','got','ur','wanted',
#                    'may', 'the', 'just', 'can', 'think', 'damn', 'still', 'guys', 'literally', 'hopefully', 'much', 'even', 'rly', 'guess', 'anon']#anything with a length of one
                   

# '''pre-processing'''
# # original_doc_set = list(df.text[df.photo=='no'])
# sel_df = df[df.photo=='no'] #extracting only-text posts
# doc_list = list(sel_df.text)


# ##############reading AP corpus
# text_df = ap_corpus('./data/ap.txt')
# doc_list = list(text_df.text_cleaned)

##############reading Newsgroup corpus
# text_df = newsgroup('./data/20newsgroup_preprocessed.csv')
# doc_list = list(text_df.text_cleaned)

##############Covid Tweet corpus
doc_list=[]
with open('./data/covid_tweets','r',encoding='utf-8') as txtfile:
    doc_list = txtfile.readlines()


#loading ref corpus for coherene score for lda_mallet
wiki_docs = loading_wiki_docs('./data/wiki_sampled_5p.txt')
#doing pre-processing on wiki-pedia documents
pre_processed_wiki, _ = preprocess_data(wiki_docs)
wiki_vocab_dict, _ = prepare_corpus(pre_processed_wiki)
del wiki_docs

In [54]:
#removing terms that are not in Wikipedia ref-corpus
pre_processed_docs,filtered_docs = preprocess_data(doc_list,extra_stopwords={})
#generate vocabulary and texts
vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_docs)

#finding stopwords that are not in Wikipedia and removing those
extra_stopwords = set(vocab_dict.token2id.keys()).difference(set(wiki_vocab_dict.token2id.keys()))
pre_processed_docs,filtered_docs = preprocess_data(doc_list,extra_stopwords=extra_stopwords)
vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_docs)

In [55]:
#running many many LDA and storing their pair of terms

start = 36; limit=68; step = 1
runs = 3
term_pairs = set()

for num_topics in tqdm(range(start, limit+1, step)):
    model_t = []
    purity_t = []
    coherence_t = []
    contrast_t = []
    for r in range(runs):
        #model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=num_topics, id2word=vocab_dict,optimize_interval = 25,workers=1)

        #storing top_terms
        for tn in range(num_topics): 
            tt = model.show_topic(tn,topn=20)

            #saving top_terms and their counts
            top_terms = [i[0] for i in tt]

            #making pair terms
            for i in range(len(top_terms)):
                for j in range(i+1,len(top_terms)):
                    term_pairs.add((top_terms[i],top_terms[j]))
    

  0%|                                                    | 0/33 [00:00<?, ?it/s]Mallet LDA: 36 topics, 6 topic bits, 111111 topic mask
Data loaded.
max tokens: 35
total tokens: 3088644
<10> LL/token: -10.72143
<20> LL/token: -9.83314
<30> LL/token: -9.17704
<40> LL/token: -8.78943

0	1.38889	lives life cancer vulnerable issue spreading illness transmission shows analysis normal severe die prevent vaccines misinformation problems investigation folks release 
1	1.38889	analytics latest data team county insights population confirmed day growth deaths cases daily distribution total state tn dc fl wa 
2	1.38889	care workers hiv supply health wastewater gop access dr water highly mental big sites surveillance issues provide helping tax nose 
3	1.38889	health coronavirus today patients corona vaccine policy medicine doctor war nhs hospital insurance climate pharmaceutical biotech solution medical hospitals treatment 
4	1.38889	china cases shanghai reported infections city local chinese lockdo

<100> LL/token: -8.31774
<110> LL/token: -8.30308
<120> LL/token: -8.28948
<130> LL/token: -8.27958
<140> LL/token: -8.27221

0	1.38889	life lives vulnerable lost back spreading normal save left wrong heart line indian matter giving times failed folks investigation humanity 
1	1.38889	analytics data latest team county insights population confirmed day growth cases deaths distribution daily total state mn fl wa nc 
2	1.38889	support access hiv policies supply work communities wastewater issues provide businesses water surveillance investment employees network helping needed tax police 
3	1.38889	today patients coronavirus health corona hospital policy medicine doctor war nhs vaccine care hospitals insurance climate pharmaceutical biotech medical solution 
4	1.38889	china cases shanghai reported infections city local chinese lockdown beijing outbreak confirmed monday wednesday thursday tuesday morning asymptomatic sunday thailand 
5	1.38889	vaccination vaccine doses pm age india vaccines

<200> LL/token: -8.23995
<210> LL/token: -8.23667
<220> LL/token: -8.23546
[beta: 0.00811] 
<230> LL/token: -8.1978
<240> LL/token: -8.19778

0	1.2539	life lives back spreading normal lost left save wrong line act matter times giving fast failed reason investigation folks humanity 
1	1.24191	analytics data latest team county insights population confirmed day growth cases deaths distribution daily total state wa fl id nc 
2	1.23729	support access work communities issues hiv policies supply provide wastewater vulnerable needed shows water helping continues problems small network tax 
3	1.14899	coronavirus patients today health corona hospital policy medicine doctor war nhs vaccine hospitals insurance climate pharmaceutical biotech care doctors medical 
4	1.19736	china shanghai cases reported city infections local chinese lockdown outbreak beijing monday wednesday tuesday thursday asymptomatic confirmed sunday amid national 
5	1.16573	vaccination vaccine doses pm india age st administered

[beta: 0.00712] 
<300> LL/token: -8.13181
<310> LL/token: -8.13291
<320> LL/token: -8.14101
[beta: 0.00722] 
<330> LL/token: -8.11648
<340> LL/token: -8.12598

0	0.75966	life lives back normal big left lost save spreading wrong times stop act line govt matter misinformation control point folks 
1	0.60775	analytics data latest team county insights population confirmed day cases deaths growth daily distribution total state death wa fl ny 
2	0.69696	support access work communities hiv policies issues supply vulnerable provide funding services affected continues helping wastewater needed investment small network 
3	0.49381	coronavirus patients today health hospital policy medicine corona doctor war nhs vaccine hospitals insurance pharmaceutical biotech care climate medical healthcare 
4	0.60445	china shanghai cases city reported local lockdown chinese infections beijing outbreak confirmed wednesday asymptomatic monday tuesday amid authorities saturday thursday 
5	0.54541	vaccination vaccin

[beta: 0.00788] 
<400> LL/token: -8.09154
<410> LL/token: -8.09369
<420> LL/token: -8.10389
[beta: 0.00826] 
<430> LL/token: -8.08057
<440> LL/token: -8.09459

0	0.42423	life lives made big normal stop save left back lost times wrong control matter good country spread misinformation govt point 
1	0.2446	analytics data latest team county insights population confirmed cases day deaths growth daily distribution total state death fl nc ny 
2	0.34914	support access work communities services hiv supply funding vulnerable digital policies provide part program including affected needed ensure monitoring response 
3	0.19442	patients health coronavirus hospital medicine today doctor nhs policy corona vaccine war insurance pharmaceutical biotech hospitals healthcare medical climate care 
4	0.27547	china shanghai cases lockdown city reported local chinese confirmed beijing infections outbreak asymptomatic amid residents tuesday taiwan monday authorities commission 
5	0.2388	vaccination vaccine dos

...

[beta: 0.0163] 
<700> LL/token: -7.87936
<710> LL/token: -7.86048
<720> LL/token: -7.85939
[beta: 0.01717] 
<730> LL/token: -7.83478
<740> LL/token: -7.83434

0	0.02272	al mohammed imam free nasser vaccination today vaccine testing visit pm clinic mahdi walk find appointment amp call information book 
1	0.05566	people amp make time good pandemic work things back making world great life made years lot point hope coming thing 
2	0.05121	health amp care pandemic healthcare support workers access people public medical patients community work women services staff communities system systems 
3	0.05408	amp pandemic join research future global response learn work lessons report today community read impact world post vaccine digital social 
4	0.00857	world ukraine healthcare coronavirus russia amp pandemic wave ehr wait emr hope light final horizon survival beat emerges simon awakened 
5	0.0235	latest daily health news update dashboard buy coronavirus straight click putting hustle dba cent doll

[beta: 0.01951] 
<800> LL/token: -7.78624
<810> LL/token: -7.77126
<820> LL/token: -7.77152
[beta: 0.02013] 
<830> LL/token: -7.75604
<840> LL/token: -7.75389

0	0.01522	al vaccine vaccination mohammed imam nasser free testing today booster visit pm clinic mahdi walk appointment find amp book call 
1	0.03804	people pandemic time amp make good back work things world making years life great made lot long flu normal coming 
2	0.03401	health amp care pandemic healthcare support workers people access public patients community work children services communities medical staff vaccines women 
3	0.03992	amp pandemic health research global response join future learn impact report work lessons read world today public support vaccine social 
4	0.00466	world ukraine healthcare coronavirus russia amp pandemic wave ehr wait emr hope horizon final survival light beat emerges simon awakened 
5	0.01607	latest daily news health update dashboard buy straight hustle dba putting coronavirus cent dollars rew

[beta: 0.02172] 
<900> LL/token: -7.7246
<910> LL/token: -7.71651
<920> LL/token: -7.71512
[beta: 0.02217] 
<930> LL/token: -7.70782
<940> LL/token: -7.70542

0	0.01152	al vaccine mohammed imam nasser vaccination booster testing free today visit clinic mahdi pm walk appointment find amp call book 
1	0.02892	people pandemic amp time make good back work world life things years virus making flu long great live lot normal 
2	0.02511	health amp care pandemic healthcare support people workers access patients community public work vaccines children medical services communities staff women 
3	0.03269	amp pandemic health research response global future impact join report work learn world public lessons read support today vaccine social 
4	0.00332	ukraine world healthcare coronavirus russia amp pandemic wave ehr emr wait final hope survival horizon light beat emerges simon obstacle 
5	0.01226	latest daily health news update buy dashboard straight hustle dba coronavirus putting cent dollars rewar

[beta: 0.02326] 
<1000> LL/token: -7.6868

Total time: 7 minutes 37 seconds
Mallet LDA: 40 topics, 6 topic bits, 111111 topic mask
Data loaded.
max tokens: 35
total tokens: 3088644
 12%|████▍                                | 4/33 [1:59:01<14:22:52, 1785.27s/it]


KeyboardInterrupt: 

In [56]:
#converting set to list
term_pairs_ls = [[i[0],i[1]] for i in list(term_pairs)]

In [58]:
#computing Coherence for all term pairs
cscore = CoherenceModel(topics=term_pairs,dictionary=wiki_vocab_dict,texts=pre_processed_wiki,coherence='c_npmi',processes=3,topn=2).get_coherence_per_topic()


In [61]:
#Loading the DB
npmi_db = db('./db/wiki_5p_old')


Load NPMI coherence DB. 
Number of keys : 213440


In [60]:
#writing into the DB
for i in tqdm(range(len(term_pairs_ls))):
    #if the key does not exist, insert it
    try:
        npmi_db.db[(term_pairs_ls[i][0],term_pairs_ls[i][1])]
    except:
        npmi_db.db[(term_pairs_ls[i][0],term_pairs_ls[i][1])] = cscore[i]
    #do the other combination
    try:
        npmi_db.db[(term_pairs_ls[i][1],term_pairs_ls[i][0])]
    except:
        npmi_db.db[(term_pairs_ls[i][1],term_pairs_ls[i][0])] = cscore[i]

100%|████████████████████████████████████| 34642/34642 [00:39<00:00, 878.90it/s]


In [28]:
# for k in tqdm(npmi_db.db.iterkeys()):
#     try:
#         npmi_db.db[(k[1],k[0])]
#     except:
#         npmi_db.db[(k[1],k[0])] =  npmi_db.db[k]

125087it [00:23, 5268.58it/s]
